In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1147173,2021-04-03,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1147174,2021-04-04,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1147175,2021-04-05,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1147176,2021-04-06,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
30815,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30817,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30819,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30821,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30823,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
707638,2021-04-03,Arkansas,Arkansas,5001,2079,36.00,5000,Arkansas,AR,Arkansas,State,3017804
707640,2021-04-04,Arkansas,Arkansas,5001,2079,36.00,5000,Arkansas,AR,Arkansas,State,3017804
707642,2021-04-05,Arkansas,Arkansas,5001,2081,36.00,5000,Arkansas,AR,Arkansas,State,3017804
707644,2021-04-06,Arkansas,Arkansas,5001,2081,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1147173,2021-04-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1147174,2021-04-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1147175,2021-04-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1147176,2021-04-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-07') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
442,2021-04-07,Snohomish,Washington,33271,563.00,53061,WA,County,822083,4047.16,68.48
882,2021-04-07,Cook,Illinois,506828,10271.00,17031,IL,County,5150233,9840.88,199.43
1321,2021-04-07,Orange,California,267253,4791.00,6059,CA,County,3175692,8415.58,150.86
1759,2021-04-07,Maricopa,Arizona,526890,9694.00,4013,AZ,County,4485414,11746.74,216.12
2197,2021-04-07,Los Angeles,California,1223832,23340.00,6037,CA,County,10039107,12190.65,232.49
...,...,...,...,...,...,...,...,...,...,...,...
1146593,2021-04-07,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1146770,2021-04-07,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1146916,2021-04-07,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1147058,2021-04-07,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
442,2021-04-07,Snohomish,Washington,33271,563.00,53061,WA,County,822083,4047.16,68.48,68.48,4047.16
882,2021-04-07,Cook,Illinois,506828,10271.00,17031,IL,County,5150233,9840.88,199.43,199.43,9840.88
1321,2021-04-07,Orange,California,267253,4791.00,6059,CA,County,3175692,8415.58,150.86,150.86,8415.58
1759,2021-04-07,Maricopa,Arizona,526890,9694.00,4013,AZ,County,4485414,11746.74,216.12,216.12,11746.74
2197,2021-04-07,Los Angeles,California,1223832,23340.00,6037,CA,County,10039107,12190.65,232.49,232.49,12190.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146593,2021-04-07,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1146770,2021-04-07,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1146916,2021-04-07,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1147058,2021-04-07,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
739893,2021-04-07,Hale,Alabama,2184,75.00,1065,AL,County,14651,14906.83,511.91,511.91,14906.83,1
805263,2021-04-07,Clarke,Alabama,3457,60.00,1025,AL,County,23622,14634.66,254.00,254.00,14634.66,2
579081,2021-04-07,Lowndes,Alabama,1362,53.00,1085,AL,County,9726,14003.70,544.93,544.93,14003.70,3
430986,2021-04-07,Franklin,Alabama,4191,82.00,1059,AL,County,31362,13363.31,261.46,261.46,13363.31,4
513520,2021-04-07,Etowah,Alabama,13654,345.00,1055,AL,County,102268,13351.19,337.35,337.35,13351.19,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1088637,2021-04-07,Platte,Wyoming,611,11.00,56031,WY,County,8393,7279.88,131.06,131.06,7279.88,19
706145,2021-04-07,Converse,Wyoming,985,17.00,56009,WY,County,13822,7126.32,122.99,122.99,7126.32,20
877610,2021-04-07,Lincoln,Wyoming,1375,12.00,56023,WY,County,19830,6933.94,60.51,60.51,6933.94,21
976516,2021-04-07,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
579081,2021-04-07,Lowndes,Alabama,1362,53.00,1085,AL,County,9726,14003.70,544.93,544.93,14003.70,3,1
739893,2021-04-07,Hale,Alabama,2184,75.00,1065,AL,County,14651,14906.83,511.91,511.91,14906.83,1,2
246483,2021-04-07,Walker,Alabama,7076,274.00,1127,AL,County,63521,11139.62,431.35,431.35,11139.62,28,3
624438,2021-04-07,Greene,Alabama,906,34.00,1063,AL,County,8111,11170.02,419.18,419.18,11170.02,27,4
578325,2021-04-07,Crenshaw,Alabama,1493,57.00,1041,AL,County,13772,10840.84,413.88,413.88,10840.84,32,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622929,2021-04-07,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
877610,2021-04-07,Lincoln,Wyoming,1375,12.00,56023,WY,County,19830,6933.94,60.51,60.51,6933.94,21,20
855780,2021-04-07,Uinta,Wyoming,2140,12.00,56041,WY,County,20226,10580.44,59.33,59.33,10580.44,4,21
245328,2021-04-07,Teton,Wyoming,3645,9.00,56039,WY,County,23464,15534.44,38.36,38.36,15534.44,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,17,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,17,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,17,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,17,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4047.16,17,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144478,2021-04-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1144479,2021-04-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1144480,2021-04-05,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1144481,2021-04-06,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
736825,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14906.83,2,1,1.00
736826,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14906.83,2,1,0.00
736827,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14906.83,2,1,0.00
736828,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14906.83,2,1,0.00
736829,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,511.91,14906.83,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892254,2021-04-03,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00
892255,2021-04-04,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00
892256,2021-04-05,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00
892257,2021-04-06,Crook,Wyoming,424,12.00,56011,WY,County,7584,5590.72,158.23,158.23,5590.72,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
576761,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14003.70,1,3,1.00,0.00
576762,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14003.70,1,3,0.00,0.00
576763,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14003.70,1,3,0.00,0.00
576764,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14003.70,1,3,0.00,0.00
576765,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14003.70,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574866,2021-04-03,Albany,Wyoming,3984,11.00,56001,WY,County,38880,10246.91,28.29,28.29,10342.08,23,5,0.00,0.00
574867,2021-04-04,Albany,Wyoming,3984,11.00,56001,WY,County,38880,10246.91,28.29,28.29,10342.08,23,5,0.00,0.00
574868,2021-04-05,Albany,Wyoming,3990,11.00,56001,WY,County,38880,10262.35,28.29,28.29,10342.08,23,5,6.00,0.00
574869,2021-04-06,Albany,Wyoming,4006,11.00,56001,WY,County,38880,10303.50,28.29,28.29,10342.08,23,5,16.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-07') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
294191,2021-04-07,Imperial,California,27661,712.00,6025,CA,County,181215,15264.19,392.90,392.90,15264.19,1,2,57.00,1.00
2197,2021-04-07,Los Angeles,California,1223832,23340.00,6037,CA,County,10039107,12190.65,232.49,232.49,12190.65,2,5,397.00,38.00
146579,2021-04-07,San Bernardino,California,291989,4428.00,6071,CA,County,2180085,13393.47,203.11,203.11,13393.47,3,4,261.00,3.00
514988,2021-04-07,Inyo,California,1379,36.00,6027,CA,County,18039,7644.55,199.57,199.57,7644.55,4,25,2.00,0.00
65589,2021-04-07,Stanislaus,California,59603,1009.00,6099,CA,County,550660,10823.92,183.23,183.23,10823.92,5,9,67.00,2.00
30417,2021-04-07,Riverside,California,295770,4442.00,6065,CA,County,2470546,11971.85,179.80,179.80,11971.85,6,6,139.00,25.00
81306,2021-04-07,Tulare,California,49188,819.00,6107,CA,County,466195,10550.95,175.68,175.68,10550.95,7,10,13.00,1.00
52195,2021-04-07,San Joaquin,California,70511,1325.00,6077,CA,County,762148,9251.62,173.85,173.85,9251.62,8,19,46.00,0.00
29623,2021-04-07,Fresno,California,99814,1626.00,6019,CA,County,999101,9990.38,162.75,162.75,9990.38,9,14,59.00,4.00
466745,2021-04-07,Merced,California,30968,442.00,6047,CA,County,277680,11152.41,159.18,159.18,11152.41,10,8,29.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1088188,2021-04-07,Lassen,California,5660,1.00,6035,CA,County,30573,18513.07,3.27,3.27,18513.07,56,1,0.00,0.00
294191,2021-04-07,Imperial,California,27661,712.00,6025,CA,County,181215,15264.19,392.90,392.90,15264.19,1,2,57.00,1.00
624380,2021-04-07,Kings,California,22676,243.00,6031,CA,County,152940,14826.73,158.89,158.89,14826.73,11,3,2.00,1.00
146579,2021-04-07,San Bernardino,California,291989,4428.00,6071,CA,County,2180085,13393.47,203.11,203.11,13393.47,3,4,261.00,3.00
2197,2021-04-07,Los Angeles,California,1223832,23340.00,6037,CA,County,10039107,12190.65,232.49,232.49,12190.65,2,5,397.00,38.00
30417,2021-04-07,Riverside,California,295770,4442.00,6065,CA,County,2470546,11971.85,179.80,179.80,11971.85,6,6,139.00,25.00
184257,2021-04-07,Kern,California,106628,1293.00,6029,CA,County,900202,11844.90,143.63,143.63,11844.90,14,7,85.00,1.00
466745,2021-04-07,Merced,California,30968,442.00,6047,CA,County,277680,11152.41,159.18,159.18,11152.41,10,8,29.00,0.00
65589,2021-04-07,Stanislaus,California,59603,1009.00,6099,CA,County,550660,10823.92,183.23,183.23,10823.92,5,9,67.00,2.00
81306,2021-04-07,Tulare,California,49188,819.00,6107,CA,County,466195,10550.95,175.68,175.68,10550.95,7,10,13.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
294191,2021-04-07,Imperial,California,27661,712.00,6025,CA,County,181215,15264.19,392.90,392.90,15264.19,1,2,57.00,1.00
2197,2021-04-07,Los Angeles,California,1223832,23340.00,6037,CA,County,10039107,12190.65,232.49,232.49,12190.65,2,5,397.00,38.00
146579,2021-04-07,San Bernardino,California,291989,4428.00,6071,CA,County,2180085,13393.47,203.11,203.11,13393.47,3,4,261.00,3.00
514988,2021-04-07,Inyo,California,1379,36.00,6027,CA,County,18039,7644.55,199.57,199.57,7644.55,4,25,2.00,0.00
65589,2021-04-07,Stanislaus,California,59603,1009.00,6099,CA,County,550660,10823.92,183.23,183.23,10823.92,5,9,67.00,2.00
30417,2021-04-07,Riverside,California,295770,4442.00,6065,CA,County,2470546,11971.85,179.80,179.80,11971.85,6,6,139.00,25.00
81306,2021-04-07,Tulare,California,49188,819.00,6107,CA,County,466195,10550.95,175.68,175.68,10550.95,7,10,13.00,1.00
52195,2021-04-07,San Joaquin,California,70511,1325.00,6077,CA,County,762148,9251.62,173.85,173.85,9251.62,8,19,46.00,0.00
29623,2021-04-07,Fresno,California,99814,1626.00,6019,CA,County,999101,9990.38,162.75,162.75,9990.38,9,14,59.00,4.00
466745,2021-04-07,Merced,California,30968,442.00,6047,CA,County,277680,11152.41,159.18,159.18,11152.41,10,8,29.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.90,15264.19,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.90,15264.19,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.90,15264.19,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.90,15264.19,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.90,15264.19,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3852,04/03/21,Lassen,5652,1.00,18486.90,3.27,3.27,18513.07,56,1,0.00,0.00
3853,04/04/21,Lassen,5652,1.00,18486.90,3.27,3.27,18513.07,56,1,0.00,0.00
3854,04/05/21,Lassen,5656,1.00,18499.98,3.27,3.27,18513.07,56,1,4.00,0.00
3855,04/06/21,Lassen,5660,1.00,18513.07,3.27,3.27,18513.07,56,1,4.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.90,15264.19,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.90,15264.19,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.90,15264.19,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.90,15264.19,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.90,15264.19,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3938,04/03/21,Merced,30687,439.00,11051.21,158.10,159.18,11152.41,10,8,0.00,0.00
3939,04/04/21,Merced,30687,439.00,11051.21,158.10,159.18,11152.41,10,8,0.00,0.00
3940,04/05/21,Merced,30906,442.00,11130.08,159.18,159.18,11152.41,10,8,219.00,3.00
3941,04/06/21,Merced,30939,442.00,11141.96,159.18,159.18,11152.41,10,8,33.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)